In [4]:
import pandas as pd
import sqlalchemy
from binance import Client
from binance import BinanceSocketManager

In [5]:
with open('access.txt') as f:
    acc = f.readlines()
api = acc[0].strip()
key = acc[1].strip()

In [6]:
client = Client(api, key)

In [8]:
bsm = BinanceSocketManager(client, user_timeout=10)
ts = bsm.trade_socket('DOGEUSDT')
# enter the context manager
await ts.__aenter__()
# receive a message
msg = await ts.recv()
print(msg)

{'e': 'trade', 'E': 1625633284829, 's': 'DOGEUSDT', 't': 285413428, 'p': '0.23533000', 'q': '42.50000000', 'b': 1424830616, 'a': 1424830658, 'T': 1625633284829, 'm': True, 'M': True}


In [17]:
def create_df(message):
    df = pd.DataFrame([message])
    df = df.loc[:,['s','E','p','q']]
    df.E = pd.to_datetime(df.E, unit='ms')
    df[['p','q']] =  df[['p','q']].astype('float')
    df.columns = ['Symbol','Time','Price','Quantity']
    return df

In [18]:
df = create_df(msg)

In [19]:
df.head()

,Symbol,Time,Price,Quantity
0,DOGEUSDT,2021-07-07 04:48:04.829,0.23533,42.5


In [22]:
# create sqlalchemy engine
engine = sqlalchemy.create_engine('sqlite:///DogeTrade.db')

In [23]:
for i in range(10):
    await ts.__aenter__()
    msg = await ts.recv()
    df = create_df(msg)
    df.to_sql('doge_test_1', engine, if_exists='append', index=False)